# **Taller 1 MIS**


##### Juan Felipe Agudelo, Santiago Caraballo, Rafael Santofimio, Juan Diego Valencia 

In [1]:
#Importar los paquetes
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import itertools as it
import time

### Punto 1

1. Calcule todas las posibles redes de tamaño 5 y guárdelas en un listado. Asegúrese de guardar únicamente las redes que no son isomorfas entre si, i.e. guarde únicamente un representante de cada conjunto de redes isomorfas entre si

In [2]:

# Función que permite crear las matrices de adyacencia a partir de listados de 0 y 1. 
def matrix_creator(lista,n):
    m = len(lista)
    if 2*m == (n*(n-1)):
        contador = 1
        A = []
        AA = [0 for i in range(n)]
        while contador < n+1:
            limite = n - contador
            B = lista[:limite]
            lista = lista[limite:]
            C = AA[:contador] + B
            A.append(C)
            contador+=1
        A = np.array(A)
        return A + np.transpose(A)
    else:
        print("Introduzca los inputs correctos")

In [3]:
#Funcion que genera una red de longitud n

def network_creator(n):
    A = [0]*n
    B = [1]*n
    guardar = set()
    contador = 1
    while contador<n:
        C = B[:contador]+A[contador:]
        guardar = guardar.union(set(it.permutations(C,n)))
        contador+=1
    guardar = [list(i) for i in guardar]
    guardar = [A] + guardar + [B]
    return(guardar)



In [4]:
n = 5
longitud = int(((n-1)*n)/2)
M_Adyacencia = [matrix_creator(i,n) for i in network_creator(longitud)]
Grafos = [nx.from_numpy_array(i) for i in M_Adyacencia]

In [10]:
Revisor = {i:True for i in range(len(Grafos))}
for i in Revisor:
    for j in Revisor:
        if j>i and Revisor[i] and Revisor[j] :
            if nx.is_isomorphic(Grafos[i],Grafos[j]):
                Revisor[j] = False

Grafos_Def = [Grafos[i] for i in Revisor if Revisor[i]]
M_Adyacencia_Def = [M_Adyacencia[i] for i in Revisor if Revisor[i]]
Grafos_Def

## Punto 2

Calcule la utilidad de cada nodo para cada una de las redes que creó y guarde esto como
un atributo de las redes. Utilice la función de utilidad dada por $U_i(g)=deg(i)^{1/2}$

In [9]:

# Función de utilidad que nos permite generar las utilidades relevantes
def utility_function(grafo):
    utility0 = dict(grafo.degree())
    utility = {i:(utility0[i])**0.5 for i in utility0}
    return utility
# Esta funcion crea un diccionario con llaves = nodos i y valores = U_i(grafo)

In [11]:
#Se crea objeto con las utilidades de cada nodo
Utilidades = [utility_function(i) for i in Grafos_Def]
for i in range(len(Grafos_Def)):
    nx.set_node_attributes(Grafos_Def[i], Utilidades[i], "Utility")

## Punto 3

Calcule los grafos socialmente eficientes (SE). Reporte cada uno de estos (i.e. pegue un
gráfico de cada uno en el archivo .pdf y grafíquelos en el script que entregue)